## CS310 Natural Language Processing
## Lab 6: Experiment with Context-Free Grammars


In [74]:
from collections import defaultdict
from typing import List, Dict, Tuple
from pprint import pprint
import io

### T1. Examine Grammar Rules

Open the `atis3.cfg` file and examine its content. 

Start symbol is defined in the first two lines: "# Start symbols \n Top"

Lines after the comment line "# Phrasal rules" and before "# Lexical rules" are rules for non-terminal symbols, for example: `NP -> NP PP`. 

Lines after the comment line "# Lexical rules" are rules for terminal symbols, for example: `NP -> aircraft`.

**Task**: Count the number of rules whose left-hand side symbol is `NP` (inlucding both phrasal and lexical rules).


In [75]:
NP_count = 0

with open('atis3.cfg', 'r') as f:
    ### START YOUR CODE ###
     for line in f:
        split = line.split()
        if len(split) == 0:
            continue
        if split[0] == 'NP':
            NP_count += 1
    ### END YOUR CODE ###

# Test result
print('NP rules count:', NP_count)

# You should expect to see:
# NP rules count: 276

NP rules count: 276


### T2. Parse Grammar Rules

Define the function `parse_rules` that reads a string of one grammar rule and returns a tuple of two strings: left-hand side and right-hand side of the rule.

**Note**:
- The left hand side is a `str` and the right hand side is a `tuple` of `str`. Thus, in cases of lexical rules, the right-hand side tuple will have only one element.

In [76]:
def parse_rule(rule: str) -> Tuple[str, Tuple[str]]:
    ### START YOUR CODE ###
    split = rule.split("->")
    lhs = split[0].strip()
    rhs = tuple(split[1].strip().split())
    ### END YOUR CODE ###
    return lhs, rhs

# Test result
rule1 = 'NP -> DET N'
print('rule:', rule1)
lhs, rhs = parse_rule(rule1)
print('lhs:', lhs)
print('rhs:', rhs)

print()
rule2 = 'NP -> aircraft'
print('rule:', rule2)
lhs, rhs = parse_rule(rule2)
print('lhs:', lhs)
print('rhs:', rhs)

# You should expect to see:
# rule: NP -> DET N
# lhs: NP
# rhs: ('DET', 'N')

# rule: NP -> aircraft
# lhs: NP
# rhs: ('aircraft',)

rule: NP -> DET N
lhs: NP
rhs: ('DET', 'N')

rule: NP -> aircraft
lhs: NP
rhs: ('aircraft',)


**Next**, integrate the above functions to the `CFG` class. 

**Notes**:
- In the class method `read_rules`, read the rules line by line and parse them using the `parse_rules` function.
- The class member `lhs_to_rules` is a dictionary mapping the left-hand side symbol (`str`) to the complete rule returned by `parse_rules` function.
- Similarly, `rhs_to_rules` maps the right-hand side (`tuple`) to the complete rule.

In [77]:
class CFG(object):
    def __init__(self, grammar_file: io.TextIOWrapper):
        self.rhs_to_rules = defaultdict(list)
        self.lhs_to_rules = defaultdict(list)
        self.startsymbol = None
        self.read_rules(grammar_file)

    def read_rules(self, grammar_file: io.TextIOWrapper):
        for line in grammar_file:
            line = line.strip()
            if line and not line.startswith("#"):
                if "->" in line:
                    ### START YOUR CODE ###
                    lhs, rhs = self.parse_rule(line)
                    self.lhs_to_rules[lhs].append((rhs, line))
                    self.rhs_to_rules[rhs].append((lhs, line))
                    ### END YOUR CODE ###
                else:
                    startsymbol = line.strip()
                    self.startsymbol = startsymbol
    
    def parse_rule(self, rule: str) -> Tuple[str, Tuple[str]]:
        ### START YOUR CODE ###
        # Copy from the previous cell
        split = rule.split("->")
        lhs = split[0].strip()
        rhs = tuple(split[1].strip().split())
        ### END YOUR CODE ###
        return lhs, rhs

In [78]:
# Test result
with open('atis3.cfg', 'r') as f:
    cfg = CFG(f)
    print('rhs_to_rules:', len(cfg.rhs_to_rules))
    print('lhs_to_rules:', len(cfg.lhs_to_rules))
    print('startsymbol:', cfg.startsymbol)

    print()
    print('# of "NP -> *":', len(cfg.lhs_to_rules['NP']))
    print('# of "* -> aircraft":', len(cfg.rhs_to_rules[('aircraft',)]))

    print()
    print('all rules for "* -> aircraft":')
    for rule in cfg.rhs_to_rules[('aircraft',)]:
        print(rule)

rhs_to_rules: 852
lhs_to_rules: 300
startsymbol: TOP

# of "NP -> *": 276
# of "* -> aircraft": 2

all rules for "* -> aircraft":
('AIRCRAFT', 'AIRCRAFT -> aircraft')
('NP', 'NP -> aircraft')


You can see that `rhs_to_rules` provides a convenient way to find all rules that have a specific right-hand side symbol.

---

### T3. CKY Recognition

We first implement the CKY recognition algorithm cell by cell.

**Note**
- First, the **super-diagonal** elements (directly above the diagonal elements), i.e., `table[i, i+1]`, correspond to the span of length 1 in the input sentence.
- We can retrieve their rules from the `rhs_to_rules` dictionary, using the terminal symbol as the key (in tuple of length 1).

In [79]:
tokens = 'flights from miami to cleveland .'.split()
n = len(tokens)

# Initialize the table
table = [[[] for _ in range(n+1)] for _ in range(n+1)]

# Fill the super-diagonal elements
for j in range(n):
    ### START YOUR CODE ###
    for lhs, rhs in cfg.rhs_to_rules[(tokens[j],)]:
        table[j][j+1].append(lhs)
    ### END YOUR CODE ###


# Test result
pprint(table)

# You should expect to see:
# [[[], ['FLIGHTS', 'NP'], [], [], [], [], []],
#  [[], [], ['FROM', 'PP'], [], [], [], []],
#  [[], [], [], ['NP'], [], [], []],
#  [[], [], [], [], ['TO', 'X'], [], []],
#  [[], [], [], [], [], ['CLEVELAND', 'NP'], []],
#  [[], [], [], [], [], [], ['PUN']],
#  [[], [], [], [], [], [], []]]

[[[], ['FLIGHTS', 'NP'], [], [], [], [], []],
 [[], [], ['FROM', 'PP'], [], [], [], []],
 [[], [], [], ['NP'], [], [], []],
 [[], [], [], [], ['TO', 'X'], [], []],
 [[], [], [], [], [], ['CLEVELAND', 'NP'], []],
 [[], [], [], [], [], [], ['PUN']],
 [[], [], [], [], [], [], []]]


**Next**, go through each cell `[i,j]` in the rest of the table, and find all non-terminal symbols.

**Note**
- Each table cell `[i,j]` is initialized as an empty list. 
- Go through each item in `rhs_to_rules`, and check if a rule satisfies the CKY recognition condition. If so, append the entire rule to the cell `[i,j]`.
- For simplicity, we make each cell a list of unique rules to avoid duplication.

In [80]:
# Fill the rest of the table

for j in range(2, n+1):
    for i in range(j-2, -1, -1):
        for k in range(i+1, j):
            ### START YOUR CODE ###
            for B in table[i][k]:
                for C in table[k][j]:
                    for lhs, rhs in cfg.rhs_to_rules[(B, C)]:
                        table[i][j].append(lhs)
            ### END YOUR CODE ###
        table[i][j] = list(set(table[i][j]))

In [81]:
# Test result
pprint(table[0])

# You should see the following output:
# [[],
#  ['FLIGHTS', 'NP'],
#  ['NP', 'SQBAR', 'VPBAR', 'FRAG'],
#  ['NPBAR', 'FRAGBAR', 'NP', 'SQBAR', 'VPBAR', 'FRAG'],
#  [],
#  ['NPBAR', 'FRAGBAR', 'NP', 'SQBAR', 'VPBAR', 'FRAG'],
#  ['TOP']]

[[],
 ['FLIGHTS', 'NP'],
 ['VPBAR', 'FRAG', 'NP', 'SQBAR'],
 ['FRAG', 'FRAGBAR', 'SQBAR', 'VPBAR', 'NPBAR', 'NP'],
 [],
 ['FRAG', 'FRAGBAR', 'SQBAR', 'VPBAR', 'NPBAR', 'NP'],
 ['TOP']]


**Lastly**, after the table is filled, we can check if the start symbol is in cell `[0,n]`. 

If so, the input sentence is recognized by the grammar, i.e., grammatical; otherwise not.

In [82]:
if cfg.startsymbol in table[0][n]:
    print('The input string is grammatical')
else:
    print('The input string is not grammatical')

# You should see the following output:
# The input string is grammatical

The input string is grammatical


In this case, "flights from miami to cleveland ." is grammatical. 

You can try scrambling the sentence to see if the output changes.

---

### T4. Integrate into the CKY class

Integrate the above code to the `is_grammatical` method of `CKY` class, which takes a list of tokens as input and returns a boolean value.

In [83]:
class CKY(object):
    def __init__(self, cfg: CFG):
        self.cfg = cfg

    def is_grammatical(self, tokens: List[str]) -> bool:
        ### START YOUR CODE ###
        pass
        ### END YOUR CODE ###

In [84]:
# Test CKY class
with open('atis3.cfg', 'r') as f:
    cfg = CFG(f)
    cky = CKY(cfg)
    print(cky.is_grammatical('flights from miami to cleveland .'.split()))
    print(cky.is_grammatical('miami flights cleveland from to .'.split()))

None
None


### Congratulations! 

You have implemented the bare-bones CKY algorithm for recognizing sentences using a CFG.

It is not yet a complete CKY parser, as it does not return the parse tree. In order to do that, you need to store backpointers in the table, which is a bit more complex.

For example, you can replace the table entry with a `dict` whose key is the LHS of rule, and the value is a tuple of two RHS symbols (and their cell indices).